In [1]:
!pip install -U scikit-learn 

     |████████████████████████████████| 11.1MB 4.1MB/s eta 0:00:01
     |████████████████████████████████| 307kB 71.5MB/s eta 0:00:01
     |████████████████████████████████| 17.3MB 76.7MB/s eta 0:00:01
     |████████████████████████████████| 34.5MB 67.2MB/s eta 0:00:01
ERROR: mlflow 2.10.0 has requirement pytz<2024, but you'll have pytz 2024.1 which is incompatible.
ERROR: fosforio 1.0.1 has requirement pandas==2.0.0, but you'll have pandas 2.0.3 which is incompatible.
ERROR: snowflake-ml-python 1.0.1 has requirement pandas<2,>=1.0.0, but you'll have pandas 2.0.3 which is incompatible.
ERROR: snowflake-ml-python 1.0.1 has requirement snowflake-snowpark-python<2,>=1.4.0, but you'll have snowflake-snowpark-python 1.0.0 which is incompatible.
ERROR: snowflake-ml-python 1.0.1 has requirement xgboost<2,>=1.7.3, but you'll have xgboost 2.0.3 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement Flask==2.1.1; python_version >= "3.7", but you'll have flask 3.0.3 which is incomp

In [2]:
import time
from snowflake.snowpark.session import Session
import configparser
 
import warnings
warnings.filterwarnings("ignore")
 
config = configparser.ConfigParser()
config.read("snowflake_connection.ini")
 
connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": f'{config["Snowflake"]["password"]}',
    "account": f'{config["Snowflake"]["account"]}',
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}
 
def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session
 
session = snowflake_connector(connection_parameters)

connection successful!


In [3]:
df_or = session.table("CPG_DATA_ORIGINAL").to_pandas()
df_test_or = session.table("CPG_DATA_ORIGINAL").to_pandas()

In [4]:
df_or.head()

,Date,Ware_house_ID,WH_Manager_ID,Location_type,WH_capacity_size,zone,WH_regional_zone,Refill_Requests,transport_issue,No_of_Competitor,...,electric_supply,dist_from_hub,workers_num,wh_est_year,storage_issue_reported,temp_reg_mach,approved_wh_govt_certificate,wh_breakdown,govt_check,product_wg_ton
0,2023-01-01,WH_100000,EID_50000,Urban,Small,West,Zone 6,3,1,2,...,1,91.0,29.0,NaN,13,0.0,A,5.0,15.0,17115.0
1,2023-01-01,WH_100001,EID_50001,Rural,Large,North,Zone 5,0,0,4,...,1,210.0,31.0,NaN,4,0.0,A,3.0,17.0,5074.0
2,2023-01-01,WH_100002,EID_50002,Rural,Mid,South,Zone 2,1,0,4,...,0,161.0,37.0,NaN,17,0.0,A,6.0,22.0,23137.0
3,2023-01-01,WH_100003,EID_50003,Rural,Mid,North,Zone 3,7,4,2,...,0,103.0,21.0,NaN,17,1.0,A+,3.0,27.0,22115.0
4,2023-01-01,WH_100004,EID_50004,Rural,Large,North,Zone 5,3,1,2,...,1,112.0,25.0,2009.0,18,0.0,C,6.0,24.0,24071.0


In [5]:
import pandas as pd
df_or = pd.DataFrame(df_or)
df_test_or = pd.DataFrame(df_test_or)
# Convert column names to uppercase
df_or.columns = [col.upper() for col in df_or.columns]

df_test_or.columns = [col.upper() for col in df_test_or.columns]



In [7]:
df_or
df_test_or

,DATE,WARE_HOUSE_ID,WH_MANAGER_ID,LOCATION_TYPE,WH_CAPACITY_SIZE,ZONE,WH_REGIONAL_ZONE,REFILL_REQUESTS,TRANSPORT_ISSUE,NO_OF_COMPETITOR,...,ELECTRIC_SUPPLY,DIST_FROM_HUB,WORKERS_NUM,WH_EST_YEAR,STORAGE_ISSUE_REPORTED,TEMP_REG_MACH,APPROVED_WH_GOVT_CERTIFICATE,WH_BREAKDOWN,GOVT_CHECK,PRODUCT_WG_TON
0,2023-01-01,WH_100000,EID_50000,Urban,Small,West,Zone 6,3,1,2,...,1,91.0,29.0,NaN,13,0.0,A,5.0,15.0,17115.0
1,2023-01-01,WH_100001,EID_50001,Rural,Large,North,Zone 5,0,0,4,...,1,210.0,31.0,NaN,4,0.0,A,3.0,17.0,5074.0
2,2023-01-01,WH_100002,EID_50002,Rural,Mid,South,Zone 2,1,0,4,...,0,161.0,37.0,NaN,17,0.0,A,6.0,22.0,23137.0
3,2023-01-01,WH_100003,EID_50003,Rural,Mid,North,Zone 3,7,4,2,...,0,103.0,21.0,NaN,17,1.0,A+,3.0,27.0,22115.0
4,2023-01-01,WH_100004,EID_50004,Rural,Large,North,Zone 5,3,1,2,...,1,112.0,25.0,2009.0,18,0.0,C,6.0,24.0,24071.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349996,2024-02-01,WH_124996,EID_74996,Rural,Mid,West,Zone 2,4,7,1,...,1,130.0,39.0,2005.0,23,0.0,B,21.0,36.0,37853.0
349997,2024-02-01,WH_124997,EID_74997,Urban,Large,South,Zone 5,12,4,0,...,1,147.0,27.0,2012.0,12,1.0,A,30.0,55.0,65888.0
349998,2024-02-01,WH_124998,EID_74998,Rural,Small,North,Zone 1,1,4,10,...,1,60.0,44.0,NaN,24,0.0,B,17.0,38.0,38534.0
349999,2024-02-01,WH_124999,EID_74999,Rural,Mid,West,Zone 4,3,3,4,...,1,239.0,57.0,NaN,22,0.0,B+,16.0,41.0,58351.0


In [8]:

df_date = df_or[["DATE","WARE_HOUSE_ID","LOCATION_TYPE","WH_CAPACITY_SIZE","ZONE","WH_REGIONAL_ZONE","APPROVED_WH_GOVT_CERTIFICATE"]]
df_date

df_date = df_date.iloc[:326777, :]
df_date

,DATE,WARE_HOUSE_ID,LOCATION_TYPE,WH_CAPACITY_SIZE,ZONE,WH_REGIONAL_ZONE,APPROVED_WH_GOVT_CERTIFICATE
0,2023-01-01,WH_100000,Urban,Small,West,Zone 6,A
1,2023-01-01,WH_100001,Rural,Large,North,Zone 5,A
2,2023-01-01,WH_100002,Rural,Mid,South,Zone 2,A
3,2023-01-01,WH_100003,Rural,Mid,North,Zone 3,A+
4,2023-01-01,WH_100004,Rural,Large,North,Zone 5,C
...,...,...,...,...,...,...,...
326772,2024-02-01,WH_101772,Rural,Mid,North,Zone 4,C
326773,2024-02-01,WH_101773,Rural,Large,North,Zone 6,A+
326774,2024-02-01,WH_101774,Rural,Small,North,Zone 1,A+
326775,2024-02-01,WH_101775,Rural,Mid,North,Zone 4,C


In [ ]:
#df_or100 = df_or.iloc[:100, :]
#df_or100
#df_test_or = session.table("CPG_DATA_ORIGINAL").to_pandas()

#df_or100= df_or100.reset_index()
#df_or100

In [ ]:
#df_or
#df_test_or= df_test_or.reset_index()
#df_test_or.head()
#df_or
#df_test_or

In [9]:
import pandas as pd
import numpy as np
import sklearn 

#import matplotlib.pyplot as plt
#%matplotlib inline
#import seaborn as sns


from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.simplefilter(action='ignore')

In [ ]:
# train dataset
#df = pd.read_csv("1.FMCG_data_original.csv")

# test dataset
#df_test = pd.read_csv("1.FMCG_data_original.csv")

In [10]:
df_or.nunique().sort_values(ascending=False)

PRODUCT_WG_TON                  65179
WH_MANAGER_ID                   25000
WARE_HOUSE_ID                   25000
NO_OF_RETAILERS                  9068
DIST_FROM_HUB                     217
NUMBER_OF_DISTRIBUTORS            138
WORKERS_NUM                       122
STORAGE_ISSUE_REPORTED             73
GOVT_CHECK                         70
WH_BREAKDOWN                       47
WH_EST_YEAR                        28
REFILL_REQUESTS                    27
TRANSPORT_ISSUE                    23
NO_OF_COMPETITOR                   23
DATE                               14
APPROVED_WH_GOVT_CERTIFICATE        7
WH_REGIONAL_ZONE                    6
ZONE                                4
WH_CAPACITY_SIZE                    3
LOCATION_TYPE                       2
FLOOD_PROOF                         2
FLOOD_IMPACTED                      2
TEMP_REG_MACH                       2
WAREHOUSE_OWNERSHIP                 2
ELECTRIC_SUPPLY                     2
dtype: int64

In [11]:

df_or.shape



(350001, 25)

In [12]:
#df_or.drop(columns=['WARE_HOUSE_ID','WH_MANAGER_ID','DATE'], inplace = True)
df_or['WORKERS_NUM'].median()
df_or['APPROVED_WH_GOVT_CERTIFICATE'].mode()
df_or['APPROVED_WH_GOVT_CERTIFICATE'].fillna(df_or['APPROVED_WH_GOVT_CERTIFICATE'].mode()[0], inplace=True)
df_or.isnull().sum()
df_or.nunique().sort_values(ascending= True)

ELECTRIC_SUPPLY                     2
LOCATION_TYPE                       2
FLOOD_PROOF                         2
FLOOD_IMPACTED                      2
WAREHOUSE_OWNERSHIP                 2
TEMP_REG_MACH                       2
WH_CAPACITY_SIZE                    3
ZONE                                4
WH_REGIONAL_ZONE                    6
APPROVED_WH_GOVT_CERTIFICATE        7
DATE                               14
NO_OF_COMPETITOR                   23
TRANSPORT_ISSUE                    23
REFILL_REQUESTS                    27
WH_EST_YEAR                        28
WH_BREAKDOWN                       47
GOVT_CHECK                         70
STORAGE_ISSUE_REPORTED             73
WORKERS_NUM                       122
NUMBER_OF_DISTRIBUTORS            138
DIST_FROM_HUB                     217
NO_OF_RETAILERS                  9068
WH_MANAGER_ID                   25000
WARE_HOUSE_ID                   25000
PRODUCT_WG_TON                  65179
dtype: int64

In [13]:
for i in df_or.columns[df_or.nunique()<=4]:
    print("Number of unique values in column ", i, ' :', len(df_or[i].unique()))
    print(df_or[i].value_counts())
    print(' ')

Number of unique values in column  LOCATION_TYPE  : 2
LOCATION_TYPE
Rural    321398
Urban     28603
Name: count, dtype: int64
 
Number of unique values in column  WH_CAPACITY_SIZE  : 3
WH_CAPACITY_SIZE
Large    142366
Mid      140280
Small     67355
Name: count, dtype: int64
 
Number of unique values in column  ZONE  : 4
ZONE
North    143892
West     111035
South     89068
East       6006
Name: count, dtype: int64
 
Number of unique values in column  WAREHOUSE_OWNERSHIP  : 2
WAREHOUSE_OWNERSHIP
Company Owned    190092
Rented           159909
Name: count, dtype: int64
 
Number of unique values in column  FLOOD_IMPACTED  : 2
FLOOD_IMPACTED
0    315645
1     34356
Name: count, dtype: int64
 
Number of unique values in column  FLOOD_PROOF  : 2
FLOOD_PROOF
0    330876
1     19125
Name: count, dtype: int64
 
Number of unique values in column  ELECTRIC_SUPPLY  : 2
ELECTRIC_SUPPLY
1    229909
0    120092
Name: count, dtype: int64
 
Number of unique values in column  TEMP_REG_MACH  : 3
TEMP_REG

In [14]:
def count_outliers(df_or):
    outliers={}
    for i in df_or.columns:
        if pd.api.types.is_numeric_dtype(df_or[i]):
            q1 = df_or[i].quantile(0.25)
            q3 = df_or[i].quantile(0.75)
            iqr = q3 - q1
            lower = q1 - 1.5*iqr
            upper = q3 + 1.5*iqr
            outliers[i] = df_or[(df_or[i] < lower) | (df_or[i] > upper)].shape[0]
        else:
            pass
    return outliers

In [15]:
outlier_counts = count_outliers(df_or)
outlier_counts

{'REFILL_REQUESTS': 4159,
 'TRANSPORT_ISSUE': 3852,
 'NO_OF_COMPETITOR': 16980,
 'NO_OF_RETAILERS': 7980,
 'NUMBER_OF_DISTRIBUTORS': 494,
 'FLOOD_IMPACTED': 34356,
 'FLOOD_PROOF': 19125,
 'ELECTRIC_SUPPLY': 0,
 'DIST_FROM_HUB': 0,
 'WORKERS_NUM': 8332,
 'WH_EST_YEAR': 0,
 'STORAGE_ISSUE_REPORTED': 932,
 'TEMP_REG_MACH': 0,
 'WH_BREAKDOWN': 436,
 'GOVT_CHECK': 114,
 'PRODUCT_WG_TON': 1327}

In [16]:
df_or['PRODUCT_WG_TON'].corr(df_or['FLOOD_PROOF'])
df_or['PRODUCT_WG_TON'].corr(df_or['FLOOD_IMPACTED'])
df_or.drop(columns=['FLOOD_PROOF', 'FLOOD_IMPACTED'], inplace = True)

In [17]:
def remove_outliers(df_or):
    num_df = df_or.select_dtypes(include=['int', 'float'])
    q1 = num_df.quantile(0.25)
    q3 = num_df.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5*iqr
    upper = q3 + 1.5*iqr
    df_no_outliers = num_df[((num_df >= lower) & (num_df <= upper)).all(axis=1)]
    return df_or.loc[df_no_outliers.index]

In [18]:
df_or = remove_outliers(df_or)
df_or.shape

(173446, 23)

In [19]:
df_or

,DATE,WARE_HOUSE_ID,WH_MANAGER_ID,LOCATION_TYPE,WH_CAPACITY_SIZE,ZONE,WH_REGIONAL_ZONE,REFILL_REQUESTS,TRANSPORT_ISSUE,NO_OF_COMPETITOR,...,ELECTRIC_SUPPLY,DIST_FROM_HUB,WORKERS_NUM,WH_EST_YEAR,STORAGE_ISSUE_REPORTED,TEMP_REG_MACH,APPROVED_WH_GOVT_CERTIFICATE,WH_BREAKDOWN,GOVT_CHECK,PRODUCT_WG_TON
4,2023-01-01,WH_100004,EID_50004,Rural,Large,North,Zone 5,3,1,2,...,1,112.0,25.0,2009.0,18,0.0,C,6.0,24.0,24071.0
5,2023-01-01,WH_100005,EID_50005,Rural,Small,West,Zone 1,8,0,2,...,1,152.0,35.0,2009.0,23,1.0,A+,3.0,3.0,32134.0
6,2023-01-01,WH_100006,EID_50006,Rural,Large,West,Zone 6,8,0,4,...,1,77.0,27.0,2010.0,24,0.0,B,3.0,6.0,30142.0
8,2023-01-01,WH_100008,EID_50008,Rural,Small,South,Zone 6,8,1,4,...,1,124.0,22.0,2013.0,13,1.0,A+,5.0,2.0,18082.0
10,2023-01-01,WH_100010,EID_50010,Rural,Large,North,Zone 6,7,1,3,...,1,150.0,37.0,1999.0,17,0.0,B+,4.0,6.0,21125.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349994,2024-02-01,WH_124994,EID_74994,Rural,Mid,North,Zone 4,2,3,5,...,0,71.0,46.0,2000.0,26,0.0,A,26.0,35.0,45654.0
349995,2024-02-01,WH_124995,EID_74995,Rural,Small,North,Zone 1,11,0,7,...,1,142.0,62.0,2016.0,8,0.0,B+,28.0,38.0,52057.0
349996,2024-02-01,WH_124996,EID_74996,Rural,Mid,West,Zone 2,4,7,1,...,1,130.0,39.0,2005.0,23,0.0,B,21.0,36.0,37853.0
349997,2024-02-01,WH_124997,EID_74997,Urban,Large,South,Zone 5,12,4,0,...,1,147.0,27.0,2012.0,12,1.0,A,30.0,55.0,65888.0


In [20]:
df_test_or.nunique().sort_values(ascending=False)

PRODUCT_WG_TON                  65179
WH_MANAGER_ID                   25000
WARE_HOUSE_ID                   25000
NO_OF_RETAILERS                  9068
DIST_FROM_HUB                     217
NUMBER_OF_DISTRIBUTORS            138
WORKERS_NUM                       122
STORAGE_ISSUE_REPORTED             73
GOVT_CHECK                         70
WH_BREAKDOWN                       47
WH_EST_YEAR                        28
REFILL_REQUESTS                    27
TRANSPORT_ISSUE                    23
NO_OF_COMPETITOR                   23
DATE                               14
APPROVED_WH_GOVT_CERTIFICATE        7
WH_REGIONAL_ZONE                    6
ZONE                                4
WH_CAPACITY_SIZE                    3
LOCATION_TYPE                       2
FLOOD_PROOF                         2
FLOOD_IMPACTED                      2
TEMP_REG_MACH                       2
WAREHOUSE_OWNERSHIP                 2
ELECTRIC_SUPPLY                     2
dtype: int64

In [ ]:
#df_test_or.drop(columns=['WARE_HOUSE_ID', 'WH_MANAGER_ID','DATE'], inplace = True)
#print(len(df_test_or[df_test_or.duplicated]))

In [ ]:
df_test_or

In [21]:
# print percentage of null values in each column

null_values_features = [i for i in df_test_or.columns if df_test_or[i].isnull().sum()>1]
for i in null_values_features:
    print(i, np.round(df_test_or[i].isnull().mean()*100, 2), "% missing values")

WORKERS_NUM 0.28 % missing values
WH_EST_YEAR 47.52 % missing values


In [22]:
df_test_or.columns

Index(['DATE', 'WARE_HOUSE_ID', 'WH_MANAGER_ID', 'LOCATION_TYPE',
       'WH_CAPACITY_SIZE', 'ZONE', 'WH_REGIONAL_ZONE', 'REFILL_REQUESTS',
       'TRANSPORT_ISSUE', 'NO_OF_COMPETITOR', 'NO_OF_RETAILERS',
       'WAREHOUSE_OWNERSHIP', 'NUMBER_OF_DISTRIBUTORS', 'FLOOD_IMPACTED',
       'FLOOD_PROOF', 'ELECTRIC_SUPPLY', 'DIST_FROM_HUB', 'WORKERS_NUM',
       'WH_EST_YEAR', 'STORAGE_ISSUE_REPORTED', 'TEMP_REG_MACH',
       'APPROVED_WH_GOVT_CERTIFICATE', 'WH_BREAKDOWN', 'GOVT_CHECK',
       'PRODUCT_WG_TON'],
      dtype='object')

In [23]:
#df_test_or.drop(columns=['WH_EST_YEAR'], inplace = True)
df_test_or['WORKERS_NUM'].median()
df_test_or['WORKERS_NUM'].fillna(df_test_or['WORKERS_NUM'].median(), inplace=True)
df_test_or['APPROVED_WH_GOVT_CERTIFICATE'].mode()
df_test_or['APPROVED_WH_GOVT_CERTIFICATE'].fillna(df_test_or['APPROVED_WH_GOVT_CERTIFICATE'].mode()[0], inplace=True)
df_test_or.isnull().sum()
df_test_or.nunique().sort_values(ascending= True)

ELECTRIC_SUPPLY                     2
LOCATION_TYPE                       2
FLOOD_PROOF                         2
FLOOD_IMPACTED                      2
WAREHOUSE_OWNERSHIP                 2
TEMP_REG_MACH                       2
WH_CAPACITY_SIZE                    3
ZONE                                4
WH_REGIONAL_ZONE                    6
APPROVED_WH_GOVT_CERTIFICATE        7
DATE                               14
NO_OF_COMPETITOR                   23
TRANSPORT_ISSUE                    23
REFILL_REQUESTS                    27
WH_EST_YEAR                        28
WH_BREAKDOWN                       47
GOVT_CHECK                         70
STORAGE_ISSUE_REPORTED             73
WORKERS_NUM                       122
NUMBER_OF_DISTRIBUTORS            138
DIST_FROM_HUB                     217
NO_OF_RETAILERS                  9068
WH_MANAGER_ID                   25000
WARE_HOUSE_ID                   25000
PRODUCT_WG_TON                  65179
dtype: int64

In [24]:
for i in df_test_or.columns[df_test_or.nunique()<=4]:
    print("Number of unique values in column ", i, ' :', len(df_test_or[i].unique()))
    print(df_test_or[i].value_counts())
    print(' ')

Number of unique values in column  LOCATION_TYPE  : 2
LOCATION_TYPE
Rural    321398
Urban     28603
Name: count, dtype: int64
 
Number of unique values in column  WH_CAPACITY_SIZE  : 3
WH_CAPACITY_SIZE
Large    142366
Mid      140280
Small     67355
Name: count, dtype: int64
 
Number of unique values in column  ZONE  : 4
ZONE
North    143892
West     111035
South     89068
East       6006
Name: count, dtype: int64
 
Number of unique values in column  WAREHOUSE_OWNERSHIP  : 2
WAREHOUSE_OWNERSHIP
Company Owned    190092
Rented           159909
Name: count, dtype: int64
 
Number of unique values in column  FLOOD_IMPACTED  : 2
FLOOD_IMPACTED
0    315645
1     34356
Name: count, dtype: int64
 
Number of unique values in column  FLOOD_PROOF  : 2
FLOOD_PROOF
0    330876
1     19125
Name: count, dtype: int64
 
Number of unique values in column  ELECTRIC_SUPPLY  : 2
ELECTRIC_SUPPLY
1    229909
0    120092
Name: count, dtype: int64
 
Number of unique values in column  TEMP_REG_MACH  : 3
TEMP_REG

In [ ]:
df_test_or.columns

In [25]:
df_test_or['PRODUCT_WG_TON'].corr(df_test_or['STORAGE_ISSUE_REPORTED'])

0.6887753290004746

In [26]:
df_test_or.drop(columns=['FLOOD_PROOF', 'FLOOD_IMPACTED'], inplace = True)

In [27]:
def remove_outliers(df_test_or):
    num_df = df_test_or.select_dtypes(include=['int', 'float'])
    q1 = num_df.quantile(0.25)
    q3 = num_df.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5*iqr
    upper = q3 + 1.5*iqr
    df_no_outliers = num_df[((num_df >= lower) & (num_df <= upper)).all(axis=1)]
    return df_test_or.loc[df_no_outliers.index]

In [28]:
df_test_or = remove_outliers(df_test_or)
df_test_or

,DATE,WARE_HOUSE_ID,WH_MANAGER_ID,LOCATION_TYPE,WH_CAPACITY_SIZE,ZONE,WH_REGIONAL_ZONE,REFILL_REQUESTS,TRANSPORT_ISSUE,NO_OF_COMPETITOR,...,ELECTRIC_SUPPLY,DIST_FROM_HUB,WORKERS_NUM,WH_EST_YEAR,STORAGE_ISSUE_REPORTED,TEMP_REG_MACH,APPROVED_WH_GOVT_CERTIFICATE,WH_BREAKDOWN,GOVT_CHECK,PRODUCT_WG_TON
4,2023-01-01,WH_100004,EID_50004,Rural,Large,North,Zone 5,3,1,2,...,1,112.0,25.0,2009.0,18,0.0,C,6.0,24.0,24071.0
5,2023-01-01,WH_100005,EID_50005,Rural,Small,West,Zone 1,8,0,2,...,1,152.0,35.0,2009.0,23,1.0,A+,3.0,3.0,32134.0
6,2023-01-01,WH_100006,EID_50006,Rural,Large,West,Zone 6,8,0,4,...,1,77.0,27.0,2010.0,24,0.0,B,3.0,6.0,30142.0
8,2023-01-01,WH_100008,EID_50008,Rural,Small,South,Zone 6,8,1,4,...,1,124.0,22.0,2013.0,13,1.0,A+,5.0,2.0,18082.0
10,2023-01-01,WH_100010,EID_50010,Rural,Large,North,Zone 6,7,1,3,...,1,150.0,37.0,1999.0,17,0.0,B+,4.0,6.0,21125.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349994,2024-02-01,WH_124994,EID_74994,Rural,Mid,North,Zone 4,2,3,5,...,0,71.0,46.0,2000.0,26,0.0,A,26.0,35.0,45654.0
349995,2024-02-01,WH_124995,EID_74995,Rural,Small,North,Zone 1,11,0,7,...,1,142.0,62.0,2016.0,8,0.0,B+,28.0,38.0,52057.0
349996,2024-02-01,WH_124996,EID_74996,Rural,Mid,West,Zone 2,4,7,1,...,1,130.0,39.0,2005.0,23,0.0,B,21.0,36.0,37853.0
349997,2024-02-01,WH_124997,EID_74997,Urban,Large,South,Zone 5,12,4,0,...,1,147.0,27.0,2012.0,12,1.0,A,30.0,55.0,65888.0


In [ ]:
#df_or_dt = df_or[['Date']].astype(str)
#df_or_dt.head()


#df_or_dt


In [ ]:
#df_test_or_dt = df_test_or[['Date']].astype(str)
#df_test_or_dt.head()

In [ ]:
# df_or_dt - only date column

# df_or1 - all columns except date column 

# df_test_or_dt - only date column

# df_test_or1 - all columns except date column

In [ ]:
#df_or1 = df_or.drop(['Date'], axis=1)
#df_or1.head()

In [ ]:
#df_test_or1 = df_test_or.drop(['Date'], axis=1)
#df_test_or1.head()

In [29]:
# Numerical columns & categorical columns in train dataset

num_columns_train=[i for i in df_or.columns if df_or[i].dtypes != 'O']
cat_columns_train=[i for i in df_or.columns if df_or[i].dtypes == 'O']

# Numerical columns & categorical columns in test dataset

num_columns_test=[i for i in df_test_or.columns if df_test_or[i].dtypes != 'O']
cat_columns_test=[i for i in df_test_or.columns if df_test_or[i].dtypes == 'O']

In [31]:
cat_columns_test

['DATE',
 'WARE_HOUSE_ID',
 'WH_MANAGER_ID',
 'LOCATION_TYPE',
 'WH_CAPACITY_SIZE',
 'ZONE',
 'WH_REGIONAL_ZONE',
 'WAREHOUSE_OWNERSHIP',
 'APPROVED_WH_GOVT_CERTIFICATE']

In [32]:
for i in cat_columns_train:
    df_or[i] = LabelEncoder().fit_transform(df_or[i])

for i in cat_columns_test:
    df_test_or[i] = LabelEncoder().fit_transform(df_test_or[i])

In [33]:
# display number of unique values in each column
df_or.nunique().sort_values(ascending=False)

PRODUCT_WG_TON                  55501
WH_MANAGER_ID                   24335
WARE_HOUSE_ID                   24335
NO_OF_RETAILERS                  6279
DIST_FROM_HUB                     217
NUMBER_OF_DISTRIBUTORS            115
STORAGE_ISSUE_REPORTED             72
GOVT_CHECK                         63
WORKERS_NUM                        60
WH_BREAKDOWN                       38
WH_EST_YEAR                        28
REFILL_REQUESTS                    27
TRANSPORT_ISSUE                    23
NO_OF_COMPETITOR                   23
DATE                               14
APPROVED_WH_GOVT_CERTIFICATE        7
WH_REGIONAL_ZONE                    6
ZONE                                4
WH_CAPACITY_SIZE                    3
ELECTRIC_SUPPLY                     2
TEMP_REG_MACH                       2
LOCATION_TYPE                       2
WAREHOUSE_OWNERSHIP                 2
dtype: int64

In [34]:
columns_with_more_than_two_unique_values_train = df_or.columns[ df_or.nunique()>2]

In [35]:
columns_with_more_than_two_unique_values_test = df_test_or.columns[df_test_or.nunique()>2]

In [ ]:
# df_or_dt - only date column

# df_or1 - all columns except date column 

# df_test_or_dt - only date column

# df_test_or1 - all columns except date column

In [ ]:
#df_or2 = pd.concat([df_or_dt,df_or1], ignore_index=True, sort=False)
#df_or2.head()

In [ ]:
df_or.columns

In [ ]:
#df['Location_type'] = df['Location_type'].replace([0,1])
#df

In [36]:
X_train = df_or.drop(['PRODUCT_WG_TON', 'WH_REGIONAL_ZONE'], axis=1)
Y_train = df_or['PRODUCT_WG_TON']

In [37]:
X_train.columns

Index(['DATE', 'WARE_HOUSE_ID', 'WH_MANAGER_ID', 'LOCATION_TYPE',
       'WH_CAPACITY_SIZE', 'ZONE', 'REFILL_REQUESTS', 'TRANSPORT_ISSUE',
       'NO_OF_COMPETITOR', 'NO_OF_RETAILERS', 'WAREHOUSE_OWNERSHIP',
       'NUMBER_OF_DISTRIBUTORS', 'ELECTRIC_SUPPLY', 'DIST_FROM_HUB',
       'WORKERS_NUM', 'WH_EST_YEAR', 'STORAGE_ISSUE_REPORTED', 'TEMP_REG_MACH',
       'APPROVED_WH_GOVT_CERTIFICATE', 'WH_BREAKDOWN', 'GOVT_CHECK'],
      dtype='object')

In [ ]:
# df_or_dt - only date column

# df_or1 - all columns except date column 

# df_test_or_dt - only date column

# df_test_or1 - all columns except date column

In [ ]:
#df_test_or2 = pd.concat([df_test_or_dt,df_test_or1], ignore_index=True, sort=False)

In [ ]:
#df_test_or2.head()

In [ ]:
df_test_or

In [38]:
X_test = df_test_or.drop(['PRODUCT_WG_TON','WH_REGIONAL_ZONE'], axis=1)
X_test.shape


(173976, 21)

In [39]:
X_test.columns

Index(['DATE', 'WARE_HOUSE_ID', 'WH_MANAGER_ID', 'LOCATION_TYPE',
       'WH_CAPACITY_SIZE', 'ZONE', 'REFILL_REQUESTS', 'TRANSPORT_ISSUE',
       'NO_OF_COMPETITOR', 'NO_OF_RETAILERS', 'WAREHOUSE_OWNERSHIP',
       'NUMBER_OF_DISTRIBUTORS', 'ELECTRIC_SUPPLY', 'DIST_FROM_HUB',
       'WORKERS_NUM', 'WH_EST_YEAR', 'STORAGE_ISSUE_REPORTED', 'TEMP_REG_MACH',
       'APPROVED_WH_GOVT_CERTIFICATE', 'WH_BREAKDOWN', 'GOVT_CHECK'],
      dtype='object')

In [40]:
X_test = X_test.iloc[:326777, :]
X_test.columns

Index(['DATE', 'WARE_HOUSE_ID', 'WH_MANAGER_ID', 'LOCATION_TYPE',
       'WH_CAPACITY_SIZE', 'ZONE', 'REFILL_REQUESTS', 'TRANSPORT_ISSUE',
       'NO_OF_COMPETITOR', 'NO_OF_RETAILERS', 'WAREHOUSE_OWNERSHIP',
       'NUMBER_OF_DISTRIBUTORS', 'ELECTRIC_SUPPLY', 'DIST_FROM_HUB',
       'WORKERS_NUM', 'WH_EST_YEAR', 'STORAGE_ISSUE_REPORTED', 'TEMP_REG_MACH',
       'APPROVED_WH_GOVT_CERTIFICATE', 'WH_BREAKDOWN', 'GOVT_CHECK'],
      dtype='object')

In [41]:
#Linear Regression
LRmodel = LinearRegression()

In [42]:
LRmodel.fit(X_train, Y_train)

LinearRegression()

In [43]:
LRmodel.fit(X_train, Y_train)

LR_y_predicted = LRmodel.predict(X_test)
LR_y_predicted


array([19589.38644327, 26537.3758957 , 24997.08039974, ...,
       43857.56334851, 39014.91172457, 31372.02454517])

In [44]:
predictions_location_nochange = pd.DataFrame(LR_y_predicted, columns =['PRODUCT_WG_TON_PRED_LOCATION_NOCHANGE']) 
predictions_location_nochange


,PRODUCT_WG_TON_PRED_LOCATION_NOCHANGE
0,19589.386443
1,26537.375896
2,24997.080400
3,18523.813714
4,21154.641684
...,...
173971,49145.789713
173972,31788.588928
173973,43857.563349
173974,39014.911725


In [ ]:
#ch = predictions_location_nochange.iloc[:500, :]
#ch

In [ ]:
allcols = df_test_or.iloc[:326777, :]
allcols.isnull().sum()

In [ ]:
allcols

In [ ]:
# Assuming you have a DataFrame called 'df'
columns_to_drop = ['LOCATION_TYPE', 'WH_CAPACITY_SIZE', 'ZONE','WH_REGIONAL_ZONE','WAREHOUSE_OWNERSHIP','APPROVED_WH_GOVT_CERTIFICATE']  # Replace with actual column names
allcols.drop(columns=columns_to_drop, inplace=True)


In [ ]:
allcols

In [ ]:
final_reco = pd.concat([df_date, predictions_location_nochange,allcols], axis=1, join='outer')
final_reco.isnull().sum()

final_reco= final_reco.dropna()
final_reco.isnull().sum()
final_reco

In [ ]:
final_reco

In [ ]:
df_nochange_location=session.createDataFrame(
        final_reco.values.tolist(),
        schema=final_reco.columns.tolist())

In [ ]:
df_nochange_location.write.mode("overwrite").save_as_table("NOCHANGE_LOCATION_DEMO")

In [ ]:

allcols = df_test_or.iloc[:100, :]
allcols.isnull().sum()

In [ ]:
final_reco = pd.concat([two_cols, allcols], axis=1, join='outer')
final_reco

final_reco = final_reco.iloc[:99, :]
final_reco

In [ ]:
df_nochange_location=session.createDataFrame(
        final_reco.values.tolist(),
        schema=final_reco.columns.tolist())



In [ ]:
final_reco = pd.merge(df_test_or,predictions_location_nochange,on='index',how='inner')

In [ ]:
df_test_or = df_test_or.iloc[:100, :]
df_test_or.columns

In [ ]:
df_test_or.isnull().sum()

In [ ]:
df_test_or

In [ ]:
predictions_location_nochange = predictions_location_nochange.reset_index()
predictions_location_nochange

df_test_or

In [ ]:
df_test_or = df_test_or.reset_index()
df_test_or

In [ ]:
final_reco = pd.merge(df_test_or,predictions_location_nochange,on='index',how='inner')

In [ ]:
final_reco

In [ ]:
df_nochange_location=session.createDataFrame(
        final_reco.values.tolist(),
        schema=final_reco.columns.tolist())

df_nochange_location.write.mode("overwrite").save_as_table("NOCHANGE_LOCATION_rev")

In [ ]:
df_nochange_location.write.mode("overwrite").save_as_table("NOCHANGE_LOCATION_rev")

In [ ]:
df_test_or = session.table("CPG_DATA_ORIGINAL").to_pandas()
df_test_or
df_test_or100 = df_test_or.iloc[:100, :]
df_test_or100

In [ ]:
gg= pd.concat([df_test_or100, predictions_location_nochange], axis=1, join='outer')
gg.head(100)

In [ ]:
df_nochange_location=session.createDataFrame(
        gg.values.tolist(),
        schema=gg.columns.tolist())

In [ ]:
df_nochange_location

In [ ]:
df_nochange_location.write.mode("overwrite").save_as_table("NOCHANGE_LOCATION")

In [ ]:
predictions_location_nochange = predictions_location_nochange.reset_index()
predictions_location_nochange

In [ ]:
df_test_or = session.table("CPG_DATA_ORIGINAL").to_pandas()
df_test_or

In [ ]:
df_merged = pd.concat([df_test_or, predictions_location_nochange], ignore_index=True, sort=False)

In [ ]:
df_merged

In [ ]:
df_test_or= df_test_or.reset_index()
df_test_or.head()

In [ ]:
df_test_or = df_test_or.iloc[:100, :]
df_test_or

In [ ]:
final_reco1 = pd.merge(df_test_or,predictions_location_nochange,on='index',how='inner')

In [ ]:
final_reco1

In [ ]:
df_nochange_location=session.createDataFrame(
        final_reco1.values.tolist(),
        schema=final_reco1.columns.tolist())


In [ ]:
df_nochange_location.write.mode("overwrite").save_as_table("PRED_NOCHANGE_LOCATION")

In [ ]:
#####WITH CHANGED LOCATION 

In [ ]:
!pip install -U scikit-learn

In [ ]:
import time
from snowflake.snowpark.session import Session
import configparser
 
import warnings
warnings.filterwarnings("ignore")
 
config = configparser.ConfigParser()
config.read("snowflake_connection.ini")
 
connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": f'{config["Snowflake"]["password"]}',
    "account": f'{config["Snowflake"]["account"]}',
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}
 
def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session
 
session = snowflake_connector(connection_parameters)

In [ ]:
df_CL = session.table("CPG_DATA_ORIGINAL").to_pandas()
df_test_CL = session.table("CPG_DATA_ORIGINAL").to_pandas()

In [ ]:
df_CL.head()

In [ ]:
import numpy as np

# Calculate the number of records for 'rural' and 'urban'
total_records = len(df_CL)
rural_count = int(total_records * 0.7)
urban_count = total_records - rural_count

# Create the 'location' variable
location_changed = ['rural'] * rural_count + ['urban'] * urban_count
np.random.shuffle(location_changed)  # Shuffle to mix 'rural' and 'urban' values

# Add the 'location' variable to the DataFrame
df_CL['location_changed'] = location_changed

# Save the updated DataFrame back to a CSV file
df_CL.to_csv('CPG_DATA_LOCATION_CHANGE1.csv', index=False)  # Replace 'updated_file.csv' with the desired output file name

# Print a success message
print("The 'location' variable has been added to the CSV file with 70% rural and 30% urban categories.")


In [ ]:
df_CL.head()

In [ ]:
df_test_CL = session.table("CPG_DATA_ORIGINAL").to_pandas()

In [ ]:
import numpy as np

# Calculate the number of records for 'rural' and 'urban'
total_records = len(df_test_CL)
rural_count = int(total_records * 0.7)
urban_count = total_records - rural_count

# Create the 'location' variable
location_changed = ['rural'] * rural_count + ['urban'] * urban_count
np.random.shuffle(location_changed)  # Shuffle to mix 'rural' and 'urban' values

# Add the 'location' variable to the DataFrame
df_test_CL['location_changed'] = location_changed

# Save the updated DataFrame back to a CSV file
df_test_CL.to_csv('CPG_DATA_LOCATION_CHANGE1.csv', index=False)  # Replace 'updated_file.csv' with the desired output file name

# Print a success message
print("The 'location' variable has been added to the CSV file with 70% rural and 30% urban categories.")

In [ ]:
df_test_CL

In [ ]:
df_CL.head()


In [ ]:
df_test_CL.head()

In [ ]:
df_CL= df_CL.reset_index()
df_CL.head()

In [ ]:
df_test_CL= df_test_CL.reset_index()
df_test_CL.head()

In [ ]:
import pandas as pd
import numpy as np
import sklearn 

#import matplotlib.pyplot as plt
#%matplotlib inline
#import seaborn as sns


from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.simplefilter(action='ignore')

In [ ]:
df_CL.nunique().sort_values(ascending=False)
df_CL.drop(columns=['Ware_house_ID','WH_Manager_ID','Date'], inplace = True)
df_CL['workers_num'].median()
df_CL['approved_wh_govt_certificate'].mode()
df_CL['approved_wh_govt_certificate'].fillna(df_CL['approved_wh_govt_certificate'].mode()[0], inplace=True)
df_CL.isnull().sum()
df_CL.nunique().sort_values(ascending= True)

In [ ]:
for i in df_CL.columns[df_CL.nunique()<=4]:
    print("Number of unique values in column ", i, ' :', len(df_CL[i].unique()))
    print(df_CL[i].value_counts())
    print(' ')

In [ ]:
def count_outliers(df_CL):
    outliers={}
    for i in df_CL.columns:
        if pd.api.types.is_numeric_dtype(df_CL[i]):
            q1 = df_CL[i].quantile(0.25)
            q3 = df_CL[i].quantile(0.75)
            iqr = q3 - q1
            lower = q1 - 1.5*iqr
            upper = q3 + 1.5*iqr
            outliers[i] = df_CL[(df_CL[i] < lower) | (df_CL[i] > upper)].shape[0]
        else:
            pass
    return outliers

In [ ]:
outlier_counts = count_outliers(df_CL)
outlier_counts

In [ ]:
df_CL['product_wg_ton'].corr(df_CL['flood_proof'])
df_CL['product_wg_ton'].corr(df_CL['flood_impacted'])
df_CL.drop(columns=['flood_proof', 'flood_impacted'], inplace = True)


In [ ]:
def remove_outliers(df_CL):
    num_df = df_CL.select_dtypes(include=['int', 'float'])
    q1 = num_df.quantile(0.25)
    q3 = num_df.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5*iqr
    upper = q3 + 1.5*iqr
    df_no_outliers = num_df[((num_df >= lower) & (num_df <= upper)).all(axis=1)]
    return df_CL.loc[df_no_outliers.index]

In [ ]:
df_CL = remove_outliers(df_CL)
df_CL.shape

In [ ]:
df_test_CL.nunique().sort_values(ascending=False)

In [ ]:
df_test_CL.drop(columns=['Ware_house_ID', 'WH_Manager_ID'], inplace = True)
print(len(df_test_CL[df_test_CL.duplicated]))



In [ ]:
null_values_features = [i for i in df_test_CL.columns if df_test_CL[i].isnull().sum()>1]
for i in null_values_features:
    print(i, np.round(df_test_CL[i].isnull().mean()*100, 2), "% missing values")


In [ ]:
df_test_CL.drop(columns=['wh_est_year'], inplace = True)
df_test_CL['workers_num'].median()
df_test_CL['workers_num'].fillna(df_test_CL['workers_num'].median(), inplace=True)
df_test_CL['approved_wh_govt_certificate'].mode()
df_test_CL['approved_wh_govt_certificate'].fillna(df_test_CL['approved_wh_govt_certificate'].mode()[0], inplace=True)
df_test_CL.isnull().sum()
df_test_CL.nunique().sort_values(ascending= True)

In [ ]:
for i in df_test_CL.columns[df_test_CL.nunique()<=4]:
    print("Number of unique values in column ", i, ' :', len(df_test_CL[i].unique()))
    print(df_test_CL[i].value_counts())
    print(' ')

In [ ]:
df_test_CL['product_wg_ton'].corr(df_test_CL['storage_issue_reported'])
df_test_CL.drop(columns=['flood_proof', 'flood_impacted'], inplace = True)



In [ ]:
def remove_outliers(df_test_CL):
    num_df = df_test_CL.select_dtypes(include=['int', 'float'])
    q1 = num_df.quantile(0.25)
    q3 = num_df.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5*iqr
    upper = q3 + 1.5*iqr
    df_no_outliers = num_df[((num_df >= lower) & (num_df <= upper)).all(axis=1)]
    return df_test_CL.loc[df_no_outliers.index]

In [ ]:
df_test_CL = remove_outliers(df_test_CL)
df_test_CL.head()

In [ ]:
num_columns_train=[i for i in df_CL.columns if df_CL[i].dtypes != 'O']
cat_columns_train=[i for i in df_CL.columns if df_CL[i].dtypes == 'O']

# Numerical columns & categorical columns in test dataset

num_columns_test=[i for i in df_test_CL.columns if df_test_CL[i].dtypes != 'O']
cat_columns_test=[i for i in df_test_CL.columns if df_test_CL[i].dtypes == 'O']




In [ ]:
for i in cat_columns_train:
    df_CL[i] = LabelEncoder().fit_transform(df_CL[i])

for i in cat_columns_test:
    df_test_CL[i] = LabelEncoder().fit_transform(df_test_CL[i])
    



In [ ]:
df_CL.nunique().sort_values(ascending=False)
columns_with_more_than_two_unique_values_train = df_CL.columns[df_CL.nunique()>2]
columns_with_more_than_two_unique_values_test = df_test_CL.columns[df_test_CL.nunique()>2]


In [ ]:
X_train = df_CL.drop(['product_wg_ton','wh_est_year', 'WH_regional_zone'], axis=1)
Y_train = df_CL['product_wg_ton']
X_train.columns


In [ ]:
X_test =  df_test_CL.drop(['product_wg_ton','WH_regional_zone','Date'], axis=1)
X_test.head()
X_test.columns

In [ ]:
X_train.columns

In [ ]:
X_train= X_train.reset_index()

In [ ]:
X_train.columns

In [ ]:
X_test1 = X_test.iloc[:100, :]
X_test1.columns

In [ ]:
X_train.columns

In [ ]:
X_train = X_train.drop(['level_0'], axis=1)
X_train.columns

In [ ]:
GBmodel = GradientBoostingRegressor()
GBmodel.fit(X_train,Y_train)
GB_predicted = GBmodel.predict(X_test1)
GB_predicted

In [ ]:
predictions_location_change = pd.DataFrame(GB_predicted, columns =['product_wg_ton_pred_location_change']) 
predictions_location_change


In [ ]:
predictions_location_change = predictions_location_change.reset_index()
predictions_location_change


In [ ]:
predictions_location_change.shape

In [ ]:
#df_test_CL= df_test_CL.reset_index()
#df_test_CL.head()

df_test_CL = df_test_CL.iloc[:100, :]
df_test_CL

In [ ]:
final_reco1 = pd.merge(df_test_or,predictions_location_change,on='index',how='inner')



In [ ]:
df_nochange_location=session.createDataFrame(
        final_reco1.values.tolist(),
        schema=final_reco1.columns.tolist())



In [ ]:
df_nochange_location.write.mode("overwrite").save_as_table("CPGWH_DEMO.FMCG.CPG_DATA_PRED_NOCHANGE_LOCATION0")

In [ ]:
final_reco = pd.merge(df_test_CL,predictions_location_change,on='index',how='inner')

In [ ]:
df_change_location=session.createDataFrame(
        final_reco.values.tolist(),
        schema=final_reco.columns.tolist())




In [ ]:
df_change_location.write.mode("overwrite").save_as_table("CPGWH_DEMO.FMCG.CPG_DATA_PRED_CHANGE_LOCATION12")

In [ ]:
predictions_nolocationchange = pd.DataFrame(LR_y_predicted, columns =['product_wg_ton_pred_nolocationchange']) 
predictions_nolocationchange

In [ ]:
predictions_nolocationchange = predictions_nolocationchange.reset_index()

predictions_nolocationchange

In [ ]:
final_reco = pd.merge(df_test_or,predictions_nolocationchange,on='index',how='inner')
final_reco.head()

In [ ]:
df_nochange_location=session.createDataFrame(
        final_reco.values.tolist(),
        schema=final_reco.columns.tolist())

In [ ]:
df_test_or.write.mode("overwrite").save_as_table("CPG_DATA_PRED_NOCHANGE11")


In [ ]:
final_reco.to_csv('FMCG_data_original_predictions.csv', index=False)

In [ ]:
final_reco

In [ ]:
df_test_or.write.mode("overwrite").save_as_table("CPG_DATA_PRED_NOCHANGE11")

In [ ]:

predictions_location_nochange = predictions_nolocationchange.reset_index()

predictions_location_nochange

In [ ]:
df_test_location= df_test.reset_index()
df_test_location

In [ ]:
final_reco = pd.merge(df_test,predictions_location,on='index',how='inner')

In [ ]:
df_test.write.mode("overwrite").save_as_table("CPG_DATA_PRED_NOCHANGE1")
predictions_location.to_csv('FMCG_data_original_predictions.csv', index=False)

In [ ]:
predictions_location.to_csv('FMCG_data_original_predictions.csv', index=False)

In [ ]:
Y_test = df_test['product_wg_ton']
Y_test.head()

In [ ]:
# r2 score
lr_r2 = r2_score(Y_test, LR_y_predicted).round(3)
print('R2 score:  ', lr_r2)

# root mean squared error (RMSE)
lr_rmse = np.sqrt(mean_squared_error(Y_test, LR_y_predicted)).round(3)
print('Root Mean Squared Error:  ', lr_rmse) 

# mean absolute error (MAE)
lr_mae = mean_absolute_error(Y_test, LR_y_predicted).round(3)
print('Mean Absolute Error:  ', lr_mae)

In [ ]:
DTmodel = DecisionTreeRegressor(random_state = 0)

In [ ]:

DTmodel.fit(X_train, Y_train) 

In [ ]:
DT_predicted = DTmodel.predict(X_test)

In [ ]:
DT_predicted

In [ ]:
# r2 score
dt_r2 = r2_score(Y_test, DT_predicted).round(3)
print('R2 score:  ', dt_r2)

# root mean squared error (RMSE)
dt_rmse = np.sqrt(mean_squared_error(Y_test, DT_predicted)).round(3)
print('Root Mean Squared Error:  ', dt_rmse) 

# mean absolute error (MAE)
dt_mae = mean_absolute_error(Y_test, DT_predicted).round(3)
print('Mean Absolute Error:  ', dt_mae)

In [ ]:
RFmodel = RandomForestRegressor(n_estimators = 100, max_depth = 4, random_state = 0) 
RFmodel.fit(X_train, Y_train)
RF_predicted = RFmodel.predict(X_test)
RF_predicted

In [ ]:
# r2 score
rf_r2 = r2_score(Y_test, RF_predicted).round(3)
print('R2 score:  ', rf_r2)

# root mean squared error (RMSE)
rf_rmse = np.sqrt(mean_squared_error(Y_test, RF_predicted)).round(3)
print('Root Mean Squared Error:  ', rf_rmse) 

# mean absolute error (MAE)
rf_mae = mean_absolute_error(Y_test, RF_predicted).round(3)
print('Mean Absolute Error:  ', rf_mae)

In [ ]:

GBmodel = GradientBoostingRegressor()
GBmodel.fit(X_train,Y_train)
GB_predicted = GBmodel.predict(X_test)
GB_predicted

In [ ]:
gb_r2 = r2_score(Y_test, GB_predicted).round(3)
print('R2 score:  ', gb_r2)

# root mean squared error (RMSE)
gb_rmse = np.sqrt(mean_squared_error(Y_test, GB_predicted)).round(3)
print('Root Mean Squared Error:  ', gb_rmse) 

# mean absolute error (MAE)
gb_mae = mean_absolute_error(Y_test, GB_predicted).round(3)
print('Mean Absolute Error:  ', gb_mae)

In [ ]:
ax1 = plt.subplot2grid((2, 3), (0, 0))
plt.scatter(Y_test, LR_y_predicted, color='#ffdab9', alpha=0.3, label='Actual')
plt.plot([min(Y_test), max(Y_test)], [min(Y_test), max(Y_test)], linestyle='--', color='#2f4f4f', label='Predicted')
plt.title('Linear Regression')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.legend()


ax1 = plt.subplot2grid((2, 3), (0, 1))
plt.scatter(Y_test, DT_predicted, color='#ffdab9', alpha=0.3)
plt.plot([min(Y_test), max(Y_test)], [min(Y_test), max(Y_test)], linestyle='--', color='#2f4f4f')
plt.title('Decision Tree')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')

In [ ]:
ax1 = plt.subplot2grid((2, 3), (1, 1))
plt.scatter(Y_test, GB_predicted, color='#ffdab9', alpha=0.3)  # alpha controls point transparency for better visualization
plt.plot([min(Y_test), max(Y_test)], [min(Y_test), max(Y_test)], linestyle='--',color='#2f4f4f')
plt.title('Gradient Boosting')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')

plt.show()

In [ ]:
import numpy as np

# Calculate the number of records for 'rural' and 'urban'
total_records = len(df_or)
rural_count = int(total_records * 0.7)
urban_count = total_records - rural_count

# Create the 'location' variable
location_changed = ['rural'] * rural_count + ['urban'] * urban_count
np.random.shuffle(location_changed)  # Shuffle to mix 'rural' and 'urban' values

# Add the 'location' variable to the DataFrame
df['location_changed'] = location_changed

# Save the updated DataFrame back to a CSV file
df.to_csv('CPG_DATA_ORIGINAL.csv', index=False)  # Replace 'updated_file.csv' with the desired output file name

# Print a success message
print("The 'location' variable has been added to the CSV file with 70% rural and 30% urban categories.")
